### Library import and define methods

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
# !pip install array-to-latex

from os.path import exists
import pandas as pd
import evaluate 
import json
from semantic_group.jointEM import *
import openai
import heapq
from tqdm import tqdm 
import os
from rouge_score import rouge_scorer
metric = evaluate.load("rouge")
from tqdm import tqdm
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.util import ngrams
import string
import random
import openai
import evaluate
metric = evaluate.load("rouge")

stopword_set = set(stopwords.words('english'))
%load_ext autoreload
%autoreload 2


In [ ]:
test_num = 100
eps_1 = 0.53
eps_3 = 1.25 
eps_8 = 2.22 
eps_inf = np.inf
openai.api_key = "YOUR OPENAI API KEY"
config = {
    "model": "davinci-002", 
    "max_token": 100, 
    "temperature": 0.02 
}

In [ ]:
def selectwith_public1shot(public_example, query, candidates):
    public_ref =  public_example.split("Dialogue:")[-2]
    prompt = '['
    for candidate in candidates:
        prompt += candidate + ',\n'
    prompt = prompt[:-2] + ']'
    prompt = prompt + '\nThe summary is:'
    query = query + "\nPick the most accurate summary for the dialogue with the following summary suggestions:"
    context = "Dialogue:\n"+public_ref + "Dialogue:\n"+query
    final_prompt = context + prompt

    return final_prompt  

In [ ]:
def top_k_indices(counts, k):
    # Get the indices of the k largest elements
    return heapq.nlargest(k, range(len(counts)), key=lambda i: counts[i])

In [ ]:
def calculate_rouge_scores(predictions, references):
    # Initialize the ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    # List to store ROUGE scores for each prediction-reference pair
    scores = []

    # Calculate ROUGE scores for each pair
    for pred, ref in zip(predictions, references):
        score = scorer.score(ref, pred)  # Reference is the first argument
        scores.append(score)

    return scores

In [ ]:
def average_rouge_scores(rouge_scores, multi=None):
    if not multi: 
        # Variables to accumulate ROUGE scores for averaging
        total_rouge1 = 0
        total_rouge2 = 0
        total_rougeL = 0
        num_examples = len(rouge_scores)
        
        # Accumulate the F1 scores for each metric from the collection of scores
        for score in rouge_scores:
            total_rouge1 += score['rouge1'].fmeasure
            total_rouge2 += score['rouge2'].fmeasure
            total_rougeL += score['rougeL'].fmeasure

        # Calculate the average ROUGE scores
        avg_rouge1 = total_rouge1 / num_examples
        avg_rouge2 = total_rouge2 / num_examples
        avg_rougeL = total_rougeL / num_examples
        
        print(f"Mean: rouge1: {avg_rouge1}, rouge2: {avg_rouge2}, rougeL:{avg_rougeL}")
        
    else:
        rouge1 = []
        rouge2 = []
        rougeL = []
        # In this case, rouge_scores is nested array. 
        for scores in rouge_scores:
                    # Variables to accumulate ROUGE scores for averaging
            total_rouge1 = 0
            total_rouge2 = 0
            total_rougeL = 0
            num_examples = len(scores)
            
            # Accumulate the F1 scores for each metric from the collection of scores
            for score in scores:
                total_rouge1 += score['rouge1'].fmeasure
                total_rouge2 += score['rouge2'].fmeasure
                total_rougeL += score['rougeL'].fmeasure

            # Calculate the average ROUGE scores
            avg_rouge1 = total_rouge1 / num_examples
            avg_rouge2 = total_rouge2 / num_examples
            avg_rougeL = total_rougeL / num_examples
            rouge1.append(avg_rouge1)
            rouge2.append(avg_rouge2)
            rougeL.append(avg_rougeL)
        rouge1_avg, rouge2_avg, rougeL_avg = np.mean(rouge1)*100, np.mean(rouge2)*100, np.mean(rougeL)*100
        rouge1_std, rouge2_std, rougeL_std = np.std(rouge1)*100, np.std(rouge2)*100, np.std(rougeL)*100

        print(f"Mean: rouge1: {rouge1_avg:.3f}, rouge2: {rouge2_avg:.3f}, rougeL:{rougeL_avg:.3f}")
        print(f"Std: rouge1: {rouge1_std:.3f}, rouge2: {rouge2_std:.3f}, rougeL:{rougeL_std:.3f}")



### Histogram Load 

In [ ]:
def add_countfield(histograms): 
    for i, ensemble in enumerate(histograms):
        count = []
        for k,v in ensemble.items(): 
            if k not in ('query', 'reference', 'count'): 
                count.append(len(ensemble[k]))
        histograms[i]['count'] = count 
    return histograms

In [ ]:
def histogram_extract(histograms, tag=None):
    new_histograms =[] 
    for i, ensemble in enumerate(histograms):
        new_histogram ={}
        for k,v in ensemble.items():
            if k not in ('query', 'reference', 'count'):
                new_hist =[]  
                for elem in v:
                    if elem[1] == tag:
                        new_hist.append(elem)
                if len(new_hist) > 0:
                    new_histogram[k] = new_hist 

            else:
                new_histogram[k] = v  
        new_histograms.append(new_histogram)
        
    return new_histograms

In [ ]:
def load_histogram(path):
    with open(path ,'r') as f:
        file = json.load(f)
        histograms =[]
        for line in file:
            histograms.append(line)
    return histograms

In [ ]:
path = "semantic_group/Summarization_davinci-002_100way-4shot_aggregate_090.json"
private_path = "semantic_group/Summarization_davinci-002_100way-private_090.json"
histograms = load_histogram(path)
private_histograms = load_histogram(private_path)

In [ ]:
output_private = "output/Summarization_davinci-002_100way-4shot_private.json"
output_public = "output/Summarization_davinci-002_100way-4shot_public.json"
with open(output_private, 'r') as f:
    file_private = json.load(f)
with open(output_public, 'r') as f:
    file_public = json.load(f)

### KSA

In [ ]:
# get predictions from the json file
trials = 1
ds_size = 100 
ensemble= 100 
private_only = True 
eps = eps_1
final_preds_group =[]
references_group = []
for trial in range(trials):
     random.seed(2*trial)
     np.random.seed(2*trial)
     final_preds = []
     references =[]
     for i in tqdm(range(ds_size)):
          all_tokens = {} # key: token, value: count
          for j in range(ensemble):
               sentence_private, sentence_public = file_private[i*ensemble+j]['prediction'], file_public[i*ensemble+j]['prediction']
               tokens_private = nltk.word_tokenize(sentence_private)
               tokens_public = nltk.word_tokenize(sentence_public)
               tokens_group = (tokens_private, tokens_public) if not private_only else [tokens_private]
               for tokens in tokens_group:
                    onegrams = set(ngrams(tokens, 1))
                    # onegrams = set(onegrams)
                    # making onegrams a set to avoid duplicate tokens
                    for token in onegrams:
                         # only add one gram for one sentence
                         if token in all_tokens:
                              all_tokens[token] += 1
                         else:
                              all_tokens[token] = 1
          all_tokens_sorted = sorted(all_tokens.items(), key=lambda x: x[1], reverse=True)
          # ignore those non-words tokens
          filtered_tokens = {}
          for token, count in all_tokens_sorted:
               if not all(word in string.punctuation for word in token) and token[0] not in stopword_set:
                    filtered_tokens[token] = count
          filtered_tokens_sorted = sorted(filtered_tokens.items(), key=lambda x: x[1], reverse=True)
          item_counts = np.array([count for token, count in filtered_tokens_sorted])
          joint_out = joint(item_counts, k=min(10, len(item_counts)), epsilon=eps, neighbor_type=1)
          filtered_tokens = np.array(filtered_tokens_sorted, dtype=object)[joint_out]
          filtered_tokens = [token_tuple[0][0] for token_tuple in filtered_tokens]
          prompt = '['
          for token in filtered_tokens:
               prompt += token + ', '
          prompt = prompt[:-2] + ']'
          prompt = prompt + '\nThe summary is:'
          public_ref =  file_public[i*ensemble+j]['prompt'].split("Dialogue:")[-2]
          query =  file_public[i*ensemble+j]['prompt'].split("Dialogue:")[-1]
          zero_shot_sentence = "Dialogue:\n"+public_ref + "Dialogue:\n"+query.replace('Summarize the above dialogue:','Summarize the above dialogue with the following word suggestions ranked by their frequency from high to low:') 
          final_prompt = zero_shot_sentence  + prompt
          response = openai.Completion.create(
                    engine=config['model'],
                    prompt=final_prompt,
                    max_tokens=config['max_token'],
                    temperature=config['temperature'], 
                    stop="\n" 
               )['choices'][0]['text']
          references.append(file_private[i*ensemble+j]['reference'])
          final_preds.append(response)
     final_preds_group.append(final_preds)
     references_group.append(references)

In [ ]:
scores = [] 
for i in range(trials):
    scores.append(calculate_rouge_scores(final_preds_group[i], references_group[i]))
average_rouge_scores(scores, multi=True)

### Prediction (Ours)

In [ ]:
trials = 3
k=3
ds_size = 100
ensemble= 100 
final_preds_group =[]
references_group = []
eps = eps_8
for trial in range(trials):
    random.seed(2*trial)
    np.random.seed(2*trial)
    predictions = []
    references = []
    for i in tqdm(range(test_num)): 
        # private top-k selection 
        cnts = np.array(histograms[i]['count'])
        joint_out = joint(cnts, k=k, epsilon=eps, neighbor_type=1)
        #joint_out = top_k_indices(cnts, k=k)
        candidates = [] 
        for idx in joint_out:
            keys = list(histograms[i].keys())[:-3]
            target_key = keys[idx]
            candidates.append(histograms[i][target_key][0][0])          # pick representative 

        query = histograms[i]['query']
        reference = histograms[i]['reference']
        prompt = selectwith_public1shot(file_public[i*ensemble]['prompt'], query, candidates)
        response = openai.Completion.create(
                    engine=config['model'],
                    prompt=prompt,
                    max_tokens=config['max_token'],
                    temperature=config['temperature'], 
                    stop="\n" 
                )
        prediction = response['choices'][0]['text']
        predictions.append(prediction)
        references.append(reference)
    final_preds_group.append(predictions)
    references_group.append(references)

In [ ]:
scores = [] 
for i in range(trials):
    scores.append(calculate_rouge_scores(final_preds_group[i], references_group[i]))
average_rouge_scores(scores, multi=True)

### Direct top-1 selection 

In [ ]:
trials = 5
ds_size = 100 
final_preds_group =[]
references_group = []
eps=eps_inf
k=6
for trial in range(trials):
    predictions = []
    references = []
    for i in tqdm(range(test_num)): 
        # private top-k selection 
        cnts = np.array(histograms[i]['count'])
        if eps==np.inf:
            joint_out = top_k_indices(cnts, k=k)
        else:
            joint_out = joint(cnts, k=k, epsilon=eps, neighbor_type=1)
        candidates = [] 
        for idx in joint_out:
            keys = list(histograms[i].keys())[:-3]
            target_key = keys[joint_out[0]]
            candidates.append(histograms[i][target_key][0][0])          # pick representative 
        prediction = candidates[0]
        reference = histograms[i]['reference']
        predictions.append(prediction)
        references.append(reference)
    final_preds_group.append(predictions)
    references_group.append(references)

In [ ]:
scores = [] 
for i in range(trials):
    scores.append(calculate_rouge_scores(final_preds_group[i], references_group[i]))
average_rouge_scores(scores, multi=True)

### 4-shot prediction 

In [ ]:
path = "output/samsum_davinci-002_5way-0shot_private.json"
trials=5
num_ensemble = 100
ensemble=5 
final_preds_group =[]
references_group = []
with open(path, 'r') as f:
    file = json.load(f)
num_examples = len(file)

for i in range(trials):
    predictions = []
    references = []
    for j in range(0, num_examples, ensemble):
        example = file[j+i] 
        prediction, reference = example["prediction"], example["reference"]
        predictions.append(prediction)
        references.append(reference)
    final_preds_group.append(predictions)
    references_group.append(references)    

In [ ]:
scores = [] 
for i in range(trials):
    scores.append(calculate_rouge_scores(final_preds_group[i], references_group[i]))
average_rouge_scores(scores, multi=True)